In [2]:
import cv2, torch, numpy as np
from pathlib import Path
from boxmot import BotSort
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights as W

dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
w = W.DEFAULT
model = fasterrcnn_resnet50_fpn_v2(weights=w, box_score_thresh=0.5).to(dev).eval()
prep = w.transforms()
tracker = BotSort(reid_weights=Path('osnet_x0_25_msmt17.pt'), device=dev, half=False)

cap = cv2.VideoCapture(0)
with torch.inference_mode():
    while True:
        ok, bgr = cap.read()
        if not ok: break
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        t = torch.from_numpy(rgb).permute(2,0,1).to(torch.uint8)
        out = model([prep(t).to(dev)])[0]
        s = out['scores'].cpu().numpy()
        keep = s >= 0.5
        dets = np.concatenate([out['boxes'][keep].cpu().numpy(),
                               s[keep,None],
                               out['labels'][keep,None].cpu().numpy()], 1)
        tracker.update(dets, bgr)
        tracker.plot_results(bgr, show_trajectories=True)
        cv2.imshow('BoXMOT + Torchvision', bgr)
        if cv2.waitKey(1) & 0xFF == ord('q'): break
cap.release(); cv2.destroyAllWindows()


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /Users/mikel.brostrom/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth
100%|██████████| 167M/167M [00:19<00:00, 9.00MB/s] 
2025-07-28 14:46:08.697 | INFO     | boxmot.utils.torch_utils:select_device:78 - BoxMOT v15.0.2 🚀 Python-3.11.5 torch-2.2.2CPU
2025-07-28 14:46:08.698 | INFO     | boxmot.appearance.backends.base_backend:download_model:138 - Downloading ReID weights from %s → %s
Downloading...
From: https://drive.google.com/uc?id=1sSwXSUlj4_tHZequ_iZ8w_Jh0VaRQMqF
To: /Users/mikel.brostrom/boxmot/examples/det/osnet_x0_25_msmt17.pt
100%|██████████| 3.06M/3.06M [00:00<00:00, 7.95MB/s]
2025-07-28 14:46:14.452 | SUCCESS  | boxmot.appearance.reid.registry:load_pretrained_weights:64 - Loaded pretrained weights from osnet_x0_25_msmt17.pt


KeyboardInterrupt: 